In [14]:
!pip install datasets
!pip install transformers[torch]

In [15]:
!pip install transformers
!pip install torch
!pip install huggingface_hub

In [16]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [17]:
from datasets import load_dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split

In [18]:
# Load your dataset
dataset = load_dataset('BI55/MedText')

dataset
# df = pd.read_csv('medtext_2.csv')

# #Split dataset into 80% train and 20% test
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

Found cached dataset csv (C:/Users/Nahush/.cache/huggingface/datasets/BI55___csv/BI55--MedText-e2a428e03a71a2c6/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<?, ?it/s]


DatasetDict({
    train: Dataset({
        features: ['Prompt', 'Completion'],
        num_rows: 1412
    })
})

In [ ]:
df.head()

In [19]:

from transformers.pipelines.base import AutoModel

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import AutoConfig
from datasets import load_dataset


tokenizer = AutoTokenizer.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B")
# tokenizer.pad_token = tokenizer.eos_token
# inputs = tokenizer(list(train_df['Prompt']), truncation=True, padding='max_length', max_length=100, return_tensors='pt')
# targets = tokenizer(list(train_df['Completion']), truncation=True, padding='max_length', max_length=100, return_tensors='pt')
# inputs["labels"] = targets["input_ids"]
# test_inputs = tokenizer(list(test_df['Prompt']), truncation=True, padding='max_length', max_length=100, return_tensors='pt')
# test_targets = tokenizer(list(test_df['Completion']), truncation=True, padding='max_length', max_length=100, return_tensors='pt')
# test_inputs["labels"] = test_targets["input_ids"]


model = AutoModelForCausalLM.from_pretrained(
    "Open-Orca/OpenOrca-Platypus2-13B",
    #load_in_8bit=True,
    device_map="auto",
)

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [13]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C'), WindowsPath('/Users/Nahush/anaconda3/envs/DTI5902_FinalProject/lib')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capability: 8.6.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/b

c:\Users\Nahush\anaconda3\envs\DTI5902_FinalProject\Lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
c:\Users\Nahush\anaconda3\envs\DTI5902_FinalProject\Lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: C:\Users\Nahush\anaconda3\envs\DTI5902_FinalProject did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 13107200 || all params: 13028992000 || trainable%: 0.10060026132489758


In [ ]:
import transformers


def merge_columns(example):
    example["prediction"] = example["Prompt"] + " ->: " + str(example["Completion"])
    return example

dataset['train'] = dataset['train'].map(merge_columns)
dataset['train']["prediction"][:5]

In [ ]:
dataset['train'][0]

In [ ]:
dataset = dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/1412 [00:00<?, ? examples/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.457100
2,1.484100
3,1.517700
4,1.468800
5,1.532300
6,1.540200
7,1.465800
8,1.492900
9,1.470300
10,1.438700


TrainOutput(global_step=200, training_loss=0.9970106321573258, metrics={'train_runtime': 1879.2286, 'train_samples_per_second': 1.703, 'train_steps_per_second': 0.106, 'total_flos': 4.891090178543616e+16, 'train_loss': 0.9970106321573258, 'epoch': 2.27})

In [ ]:
dataset['train']

Dataset({
    features: ['Prompt', 'Completion'],
    num_rows: 1412
})

In [ ]:
# !pip install bitsandbytes

# from bitsandbytes.optim import Adam8bit
# import bitsandbytes as bnb
# from torch import nn
# from transformers.trainer_pt_utils import get_parameter_names

# decay_parameters = get_parameter_names(model, [nn.LayerNorm])
# decay_parameters = [name for name in decay_parameters if "bias" not in name]
# optimizer_grouped_parameters = [
#     {
#         "params": [p for n, p in model.named_parameters() if n in decay_parameters],
#         "weight_decay": training_args.weight_decay,
#     },
#     {
#         "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
#         "weight_decay": 0.0,
#     },
# ]

# optimizer_kwargs = {
#     "betas": (training_args.adam_beta1, training_args.adam_beta2),
#     "eps": training_args.adam_epsilon,
# }
# optimizer_kwargs["lr"] = training_args.learning_rate
# adam_bnb_optim = bnb.optim.Adam8bit(
#     optimizer_grouped_parameters,
#     betas=(training_args.adam_beta1, training_args.adam_beta2),
#     eps=training_args.adam_epsilon,
#     lr=training_args.learning_rate,
)

In [ ]:
# training_args = TrainingArguments(
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=4,
#     gradient_checkpointing=True,
#     fp16=True,
#     **default_args,      # Don't push model to Hugging Face hub by default
# )

# small_train_df = train_df.iloc[:100]
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset= small_train_df,
#     eval_dataset=test_df,
#     optimizers=(adam_bnb_optim, None)
# )

# # Train the model
# trainer.train()

# model.save_pretrained('./llama2_finetuned')
# tokenizer.save_pretrained('./llama2_finetuned')

# # If we wanna  evaluate the model after training:
# results = trainer.evaluate()

# # Print results
# for key, value in results.items():
#     print(f"{key}: {value:.4f}")

# # If we want to use the model for predictions later:
# from transformers import pipeline

# generator = pipeline('text-generation', model='./llama2_finetuned', tokenizer=tokenizer)
# sample_prompt = "A 50-year-old male presents with shortness of breath and chest pain. What should be done?"
# print(generator(sample_prompt))